##### yes24 장르/나이/성별 베스트셀러 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
}

def get_yes24_bestsellers(category_number=None, sex=None, age=None, max_items=10):
    
    # 카테고리가 없으면 종합 베스트셀러 가져오기 - 전체: 001
    category_number = category_number or "001"
    url = f'https://www.yes24.com/Product/Category/DayBestSeller?categoryNumber={category_number}'
    
    # 성별 및 나이 조건 추가
    if sex: url += f"&sex={sex}"
    if age: url += f"&age={age}"

    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "html.parser")

    book_items = soup.select("div.itemUnit")[:max_items]
    book_list = []
    
    for item in book_items:
        try:
            rank = item.select_one(".ico.rank").text
            title_tag = item.select_one(".gd_name")
            author = item.select_one("a[href*='author=']").text
            publisher = item.select_one("a[href*='company=']").text
            date = item.select_one(".authPub.info_date").text

            if not all([rank, title_tag, author, publisher, date]):
                continue

            title = title_tag.text.strip()
            link = "https://www.yes24.com" + title_tag["href"]
            
            book_list.append({
                "순위": rank,
                "제목": title,
                "저자": author,
                "출판사": publisher,
                "출간일": date,
                "링크": link
            })
        except Exception:
            continue

    return pd.DataFrame(book_list)

In [2]:
# 국내도서 - 건강 취미, 여자, 20대 설정
health_category = "001001011"
sex = 'F'
age = 20
df_health = get_yes24_bestsellers(health_category, sex, age)
print(f"📘 국내도서 건강/취미 {age}대 {sex} 베스트셀러")
df_health

📘 국내도서 건강/취미 20대 F 베스트셀러


순위                            제목        저자        출판사        출간일  \
0   1            쿵야 레스토랑즈 초스페셜 스티커북  쿵야 레스토랑즈       다산북스  2025년 04월   
1   2              [예스리커버] 저속노화 식사법       정희원     테이스트북스  2024년 07월   
2   3    Report 03 : Sweet Delusion        미고       은행나무  2025년 03월   
3   4  Report 02 : Story of Persona        도미       은행나무  2025년 03월   
4   5                    디스 이즈 싱가포르       이광훈  테라(TERRA)  2025년 02월   
5   6              크툴루의 부름 : 수호자 룰북    샌디 피터슨        초여명  2016년 11월   
6   7                       리얼 후쿠오카       원경혜      한빛라이프  2025년 05월   
7   8                       당구에 미치다       신용인    일신서적출판사  2025년 04월   
8   9       디스 이즈 스페인 THIS IS SPAIN       전혜진  테라(TERRA)  2025년 03월   
9  10                    디스 이즈 이탈리아       전혜진  테라(TERRA)  2025년 03월   

                                              링크  
0  https://www.yes24.com/product/goods/144892635  
1  https://www.yes24.com/product/goods/129124029  
2  https://www.yes24.com/product/goods/142813809  
3  https://www.yes24.com/product/goods/142813797  
4  https://www.yes24.com/product/goods/142790747  
5   https://www.yes24.com/product/goods/33213078  
6  https://www.yes24.com/product/goods/145381613  
7  https://www.yes24.com/product/goods/144054663  
8  https://www.yes24.com/product/goods/143750403  
9  https://www.yes24.com/product/goods/143253894

In [ ]:
"""
    # ebook 전체
    categoryNumber=017

    # 종이책
    categoryNumber=001

    # 오디오북 전체
    categoryNumber=017001008

"""


